## Zillow

For the following, iterate through the steps you would take to create functions: Write the code to do the following in a jupyter notebook, test it, convert to functions, then create the file to house those functions.

You will have a `zillow.ipynb` file and a helper file for each section in the pipeline.

**Summarize Zillow Database**

- airconditioningtype: 13 unique values
    - primary key: airconditioningtypeid


- architecturalstyletype: 27 unique values
    - primary key: architecturalstyletypeid
    
    
- buildingclasstype: 5 unique values
    - primary key: buildingclasstypeid
    
    
- heatingorsystemtype: 25 unique values
    - primary key: heatingorsystemtypeid
    
    
- predictions_2016: all the transactions in 2016 
    - No need to be joined
    
    
- predictions_2017: 77614 records in total
    - primary key: parcelid
    - 77613 records in 2017
    - 1 record in 2018
    - unique id: 77614
    - **unique parcelid: 77414**
    
    
- properties_2016: No need to be joined


- properties_2017: main table
    - primary key: parcelid
    
    
- propertylandusetype
    - primary key: propertylandusetypeid
    
    
- storytype: 35 unique values
    - primary key: storytypeid
    

- typeconstructiontype: 18 unqiue values
    - primary key: typeconstructiontypeid
    
    
- unique_properties: 2,985,217 rows
    - primary key: parcelid

## acquire & summarize

### 1. Acquire data from mySQL using the python module to connect and query. You will want to end with **a single dataframe**. Make sure to include: the logerror, all fields related to the properties that are available. You will end up **using all the tables in the database**.
- Be sure to do **the correct join (inner, outer, etc.)**. We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
- Only include properties with a **transaction in 2017**, and include **only the last transaction for each properity** (so no duplicate property ID's), along with zestimate error and date of transaction.
- Only include properties that include a latitude and longitude value.

In [1]:
import warnings
warnings.filterwarnings("ignore")
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import env, acquire, summarize, prepare, wrangle_zillow

In [2]:
# Acquire properties with a transaction in 2017 order first by parcelid then transactiondate

query = """
        select *
        from properties_2017
        join predictions_2017 using(parcelid)
        left join airconditioningtype using(airconditioningtypeid)
        left join architecturalstyletype using(architecturalstyletypeid)
        left join buildingclasstype using(buildingclasstypeid)
        left join heatingorsystemtype using(heatingorsystemtypeid)
        left join propertylandusetype using(propertylandusetypeid)
        left join storytype using(storytypeid)
        left join typeconstructiontype using(typeconstructiontypeid)
        where transactiondate between '2017-01-01' and '2017-12-31'
        and latitude is not null
        and longitude is not null
        order by parcelid, transactiondate
        """

df = acquire.get_zillow_data(query, '2')
df.shape

(77579, 69)

In [3]:
df.head()

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,...,id,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,NaN,NaN,261.0,2.0,NaN,NaN,NaN,10711855,1087254,NaN,...,55006,-0.007357,2017-07-07,None,None,None,Central,Single Family Residential,None,None
1,NaN,NaN,261.0,2.0,NaN,NaN,1.0,10711877,1072280,NaN,...,71382,0.021066,2017-08-29,Central,None,None,Central,Single Family Residential,None,None
2,NaN,NaN,261.0,2.0,NaN,NaN,1.0,10711888,1340933,NaN,...,23209,0.077174,2017-04-04,Central,None,None,Central,Single Family Residential,None,None
3,NaN,NaN,261.0,2.0,NaN,NaN,NaN,10711910,1878109,NaN,...,18017,-0.041238,2017-03-17,None,None,None,Central,Single Family Residential,None,None
4,NaN,NaN,261.0,2.0,NaN,NaN,NaN,10711923,2190858,NaN,...,20378,-0.009496,2017-03-24,None,None,None,Central,Single Family Residential,None,None


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77579 entries, 0 to 77578
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   typeconstructiontypeid        223 non-null    float64
 1   storytypeid                   50 non-null     float64
 2   propertylandusetypeid         77579 non-null  float64
 3   heatingorsystemtypeid         49571 non-null  float64
 4   buildingclasstypeid           15 non-null     float64
 5   architecturalstyletypeid      207 non-null    float64
 6   airconditioningtypeid         25007 non-null  float64
 7   parcelid                      77579 non-null  int64  
 8   id                            77579 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77579 non-null  float64
 11  bedroomcnt                    77579 non-null  float64
 12  buildingqualitytypeid         49809 non-null  float64
 13  c

In [ ]:
# Address duplicates: show all duplicates

mask = df.duplicated(subset='parcelid', keep=False)
df_duplicated = df[mask]
df_duplicated.head()

In [ ]:
df_duplicated.shape

In [ ]:
# Only kee the last transaction (the most recent) for each properity. 

df.drop_duplicates(subset=['parcelid'], keep='last', inplace=True, ignore_index=True)
df.shape

In [ ]:
# Check to see whether the property with most transatction date is kept.

df[(df.parcelid == 10722858) | (df.parcelid == 10732347)]

In [ ]:
# Check if there exsits duplicate property ID

df.duplicated(subset='parcelid').any()

**Takeaways**: Properties with transaction in 2017

### 2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [5]:
zillow = prepare.keep_last_transaction(df)
zillow.shape

(77381, 69)

In [ ]:
# Summary stats

zillow.describe()

In [ ]:
# Info

zillow.info()

In [ ]:
# Display object columns and the counts of unique values

zillow_obj_sum = summarize.sum_obj_cols(zillow)
zillow_obj_sum

In [ ]:
# Count unique values in each attributes

summarize.obj_value_counts(zillow)

In [ ]:
zillow.shape

In [ ]:
zillow_num = summarize.num_df(zillow)
zillow_num.shape

In [ ]:
zillow_obj = summarize.obj_df(zillow)
zillow_obj.shape

### 3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [ ]:
zillow.head()

In [ ]:
# Compute the number of rows with missing values 

attributes_missing_values = pd.DataFrame(zillow.isna().sum(axis=0), columns=['num_row_missing'])
attributes_missing_values

In [ ]:
# Add a column to compute the percent of total rows that have missing values

total_rows = zillow.shape[0]

attributes_missing_values['pct_rows_missing'] = attributes_missing_values.num_row_missing/total_rows
attributes_missing_values.head()

In [ ]:
# Test the function

attributes_missing_values = summarize.sum_missing_values_attributes(zillow)
attributes_missing_values

### 4. Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

In [ ]:
# Count the rows based on how many missing values in that row. 

x = zillow.isnull().sum(axis=1).value_counts().sort_index()
x

In [ ]:
# Construct the dataframe from list of lists

cols_missing_values = pd.DataFrame([x.index.tolist(), x.values.tolist()], 
                                   index = ['num_cols_missing', 'num_rows'])
cols_missing_values.T

In [ ]:
# Construct the dataset from dict

d = {'num_cols_missing': x.index.tolist(), 'num_rows': x.values.tolist()}

cols_missing_values = pd.DataFrame(d)
cols_missing_values

In [ ]:
# Compute the percent of columns missing

n = zillow.shape[0] # Compuate the total number of rows
cols_missing_values['pct_cols_missing'] = (cols_missing_values.num_rows/n)*100
cols_missing_values

In [ ]:
# Visualize the distribution of the 

x = cols_missing_values.num_cols_missing
y = cols_missing_values.num_rows

plt.rc('figure', figsize=(13,7))

plt.subplot(121)
plt. bar(x, y)

plt.subplot(122)
sns.barplot(x, y)

In [ ]:
# Test the function

cols_missing_values = summarize.sum_missing_values_cols(zillow)
cols_missing_values

## Prepare
### 1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). 
- There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. 
- Add some new logic that will reduce the number of properties that are falsely removed. 
- You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

In [6]:
zillow.shape

(77381, 69)

In [ ]:
zillow.propertylandusetypeid.value_counts()

In [ ]:
zillow.propertylandusedesc.value_counts()

In [7]:
# Use the propertylandusetypeids previously used in regression project
# It is better done in the SQL

single_unit = [260, 261, 262, 279]

zillow = zillow[zillow.propertylandusetypeid.isin(single_unit)]
zillow.shape

(52357, 69)

In [8]:
# Keep homes with a postive number of bedrooms and bathrooms

zillow = zillow[(zillow.bedroomcnt > 0) & (zillow.bathroomcnt > 0)]
zillow.shape

(52179, 69)

In [9]:
zillow.head()

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,...,id,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,NaN,NaN,261.0,2.0,NaN,NaN,NaN,10711855,1087254,NaN,...,55006,-0.007357,2017-07-07,None,None,None,Central,Single Family Residential,None,None
1,NaN,NaN,261.0,2.0,NaN,NaN,1.0,10711877,1072280,NaN,...,71382,0.021066,2017-08-29,Central,None,None,Central,Single Family Residential,None,None
2,NaN,NaN,261.0,2.0,NaN,NaN,1.0,10711888,1340933,NaN,...,23209,0.077174,2017-04-04,Central,None,None,Central,Single Family Residential,None,None
3,NaN,NaN,261.0,2.0,NaN,NaN,NaN,10711910,1878109,NaN,...,18017,-0.041238,2017-03-17,None,None,None,Central,Single Family Residential,None,None
4,NaN,NaN,261.0,2.0,NaN,NaN,NaN,10711923,2190858,NaN,...,20378,-0.009496,2017-03-24,None,None,None,Central,Single Family Residential,None,None


In [14]:
# Address the null values in the unitcnt

zillow.unitcnt.value_counts(dropna=False)

1.0    33699
NaN    18451
2.0       26
4.0        2
3.0        1
Name: unitcnt, dtype: int64

In [15]:
# 18451 null unit counts. They're probably unit counts of 1.
# It's more likely the paperwork for a single unit didn't have a value

zillow.unitcnt = zillow.unitcnt.fillna(1.0)
zillow.shape

(52179, 69)

In [17]:
# How many unitcounts are not 1?

(zillow.unitcnt != 1).sum()

29

In [20]:
# Remove the duplex, triplex, etc...

zillow = zillow[zillow.unitcnt == 1.0]
zillow.shape

(52150, 69)

### 2. Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

In [ ]:
def handle_missing_values(df, prop_required_column, prop_required_row):
    """
    Drop rows and columsn based on the perent of values that are missing.
    Parameters: 
    1. df
    2. the proportion, for each column, of rows with non-missing values requied to keep the column
    3. the proportion, for each row, of columns with non-missing values required to keep the row
    """
    threshold = int(round(prop_required_column*len(df.index),0))
    df.dropna(axis=1, thresh=threshold, inplace=True)
    threshold = int(round(prop_required_row*len(df.columns),0))
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df

In [22]:
# Test the function: the columns has no more than 40% missing and the rows has no more than 40% missing

zillow_dropna = prepare.handle_missing_values(zillow, 0.6, 0.6)
zillow_dropna.shape

(52150, 35)

In [23]:
# since the inplace = True, the zillow dataset have been modified.  

zillow.shape

(52150, 35)

In [32]:
# Drop logically unnecessary columns
# 2 id cols

zillow = zillow.drop(columns=["id"])
zillow.shape

(52150, 33)

In [33]:
zillow.isnull().sum().sort_values(ascending=False)

# Then address the top 4 features in the Series below. 

buildingqualitytypeid           18537
propertyzoningdesc              18476
heatingorsystemtypeid           18334
heatingorsystemdesc             18334
regionidcity                     1028
lotsizesquarefeet                 350
finishedsquarefeet12              166
censustractandblock               109
structuretaxvaluedollarcnt         69
yearbuilt                          40
regionidzip                        23
fullbathcnt                        16
calculatedbathnbr                  16
calculatedfinishedsquarefeet        8
taxamount                           4
taxvaluedollarcnt                   1
landtaxvaluedollarcnt               1
propertylandusedesc                 0
bedroomcnt                          0
bathroomcnt                         0
parcelid                            0
fips                                0
rawcensustractandblock              0
latitude                            0
longitude                           0
propertycountylandusecode           0
regionidcoun

In [34]:
# propertyzoningdesc = Description of the allowed land uses (zoning) for that property
# I'll drop this, b/c we're already filtering for single unit residential.

zillow.drop(columns=['propertyzoningdesc'], inplace=True)
zillow.shape

(52150, 32)

In [35]:
# heatingorsystemtypeid = is the identical info for heatingorsystemdesc 
# I'll drop this

zillow.drop(columns=['heatingorsystemtypeid'], inplace=True)
zillow.shape

(52150, 31)

In [36]:
# Address the NaN in feature heatingorsystemdesc

zillow.heatingorsystemdesc.value_counts(dropna=False)

Central       20676
NaN           18334
Floor/Wall    12506
Forced air      517
Solar            85
None             16
Baseboard         7
Radiant           6
Gravity           2
Yes               1
Name: heatingorsystemdesc, dtype: int64

In [37]:
# b/c these properties are in southern california
# I'll fill the NaN with None, suggesting they don't have a/c

zillow.heatingorsystemdesc = zillow.heatingorsystemdesc.fillna("None")
zillow.heatingorsystemdesc.value_counts(dropna=False)

Central       20676
None          18350
Floor/Wall    12506
Forced air      517
Solar            85
Baseboard         7
Radiant           6
Gravity           2
Yes               1
Name: heatingorsystemdesc, dtype: int64

In [38]:
# Looks like calculatedbathrbr is equivalent to bathroomcnt

(df.calculatedbathnbr == df.bathroomcnt).sum() / len(df)

0.9921298509970148

In [39]:
zillow = zillow.drop(columns=["calculatedbathnbr"])
zillow.shape

(52150, 30)

In [51]:
# Check the status on null values

zillow.isna().sum().sort_values(ascending=False).head(10)

buildingqualitytypeid           18537
regionidcity                     1028
lotsizesquarefeet                 350
finishedsquarefeet12              166
censustractandblock               109
structuretaxvaluedollarcnt         69
yearbuilt                          40
regionidzip                        23
fullbathcnt                        16
calculatedfinishedsquarefeet        8
dtype: int64

# It's a Good Time to Split the Data
- Split the df into train/validate/test
- Impute values from train, then apply them to validate and test sets.
- Keep our out of sample datasets out of sample
### 3. Split the data into train/validate/test

In [52]:
train, validate, test = prepare.split_my_data(zillow, pct=0.15)

train.shape, validate.shape, test.shape

((31028, 30), (13299, 30), (7823, 30))

In [54]:
train.isnull().sum().sort_values(ascending=False).head(10)

buildingqualitytypeid           10958
regionidcity                      630
lotsizesquarefeet                 216
finishedsquarefeet12               93
censustractandblock                69
structuretaxvaluedollarcnt         36
yearbuilt                          23
regionidzip                        15
fullbathcnt                        10
calculatedfinishedsquarefeet        5
dtype: int64

In [55]:
# Categorical/Discrete columns to use mode to replace nulls

cols = [
    "buildingqualitytypeid",
    "regionidcity",
    "regionidzip",
    "yearbuilt",
    "regionidcity",
    "censustractandblock"
]

for col in cols:
    mode = int(train[col].mode()) 
    # I had some friction when this returned a float (and there were no decimals anyways)
    train[col].fillna(value=mode, inplace=True)
    validate[col].fillna(value=mode, inplace=True)
    test[col].fillna(value=mode, inplace=True)

In [56]:
train.isnull().sum().sort_values(ascending=False).head(10)

lotsizesquarefeet               216
finishedsquarefeet12             93
structuretaxvaluedollarcnt       36
fullbathcnt                      10
calculatedfinishedsquarefeet      5
taxamount                         3
taxvaluedollarcnt                 1
landtaxvaluedollarcnt             1
buildingqualitytypeid             0
propertycountylandusecode         0
dtype: int64

In [57]:
# Continuous valued columns to use median to replace nulls

cols = [
    "structuretaxvaluedollarcnt",
    "taxamount",
    "taxvaluedollarcnt",
    "landtaxvaluedollarcnt",
    "structuretaxvaluedollarcnt",
    "finishedsquarefeet12",
    "calculatedfinishedsquarefeet",
    "fullbathcnt",
    "lotsizesquarefeet"
]


for col in cols:
    median = train[col].median()
    train[col].fillna(median, inplace=True)
    validate[col].fillna(median, inplace=True)
    test[col].fillna(median, inplace=True)

In [60]:
train.isnull().sum().sort_values(ascending=False)

propertylandusedesc             0
heatingorsystemdesc             0
parcelid                        0
bathroomcnt                     0
bedroomcnt                      0
buildingqualitytypeid           0
calculatedfinishedsquarefeet    0
finishedsquarefeet12            0
fips                            0
fullbathcnt                     0
latitude                        0
longitude                       0
lotsizesquarefeet               0
propertycountylandusecode       0
rawcensustractandblock          0
regionidcity                    0
regionidcounty                  0
regionidzip                     0
roomcnt                         0
unitcnt                         0
yearbuilt                       0
structuretaxvaluedollarcnt      0
taxvaluedollarcnt               0
assessmentyear                  0
landtaxvaluedollarcnt           0
taxamount                       0
censustractandblock             0
logerror                        0
transactiondate                 0
propertylandus

In [61]:
# How many nulls left in the dataset?
train.isnull().sum().sort_values(ascending=False).sum()

0

In [62]:
train.shape, validate.shape, test.shape

((31028, 30), (13299, 30), (7823, 30))

### 4. Generate and test wrangle_zillow.py

In [2]:
zillow = acquire.get_zillow_clustering()
zillow.shape

# Pass since the shape matches. 

(77579, 69)

In [3]:
train, validate, test = wrangle_zillow.wrangle_zillow_clustering(zillow)

train.shape, validate.shape, test.shape

# Pass the shape matching test

((31028, 30), (13299, 30), (7823, 30))

In [5]:
train.isna().sum().sum(), validate.isna().sum().sum(), test.isna().sum().sum()

# Pass the 0-null test

(0, 0, 0)

### The wrangling function works as it should be. I'm ready to move forward to EXPLORATION. 